In [7]:
import json
import numpy as np
import os
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

In [8]:
df = np.load('../qm9_filtered.npy', allow_pickle = True)

In [9]:
df_X = []
df_y = []

for line in df:
    num = len(line['chiral_centers'])
    if num != 1:
        continue
    
    deg = line['rotation'][1]
    if deg > 0:
        sign = 1
    elif deg < 0:
        sign = 0
    else:
        continue
    
    df_X.append(line['xyz'].flatten())
    df_y.append(sign)
    
df_X = np.array(df_X)
df_y = np.array(df_y)

In [10]:
print(len(df_X), len(df_y))

22923 22923


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size = 0.2, random_state=120)

In [20]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.ensemble import RandomForestClassifier

# rfc = RandomForestClassifier()
# # number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 10, num = 5)]

# # number of features at every split
# max_features = ['auto', 'sqrt']

# # max depth
# max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
# max_depth.append(None)

# # create random grid
# random_grid = {
#     'n_estimators': n_estimators,
#     'max_features': max_features,
#     'max_depth': max_depth,
#     'min_samples_leaf': min_samples_leaf
#  }

# # Random search of parameters
# rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=40, n_jobs = -1)

# # Fit the model
# rfc_random.fit(X_train, y_train)

# # print results
# print(rfc_random.best_params_)

In [21]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 1200, min_samples_leaf = 2, max_features = 'sqrt', max_depth = 100, random_state=40)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

y_train_pred = clf.predict(X_train)

print("Train acc: ", accuracy_score(y_train, y_train_pred))
print("test acc: ", accuracy_score(y_test, y_pred))
print("f1: ", f1_score(y_test, y_pred))

Train acc:  0.9998364052786564
test acc:  0.7149400218102508
f1:  0.6922533553096304


In [22]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[1808,  599],
       [ 708, 1470]])